In [20]:
import requests
import json
import pyaudio
import random

In [3]:
HOST = "127.0.0.1"
PORT = "50021"

In [ ]:
def get_speakers():
    url = f"http://{HOST}:{PORT}/speakers"  # VOICEVOX APIのエンドポイント
    response = requests.get(url)

    speakers_dict = {}
    if response.status_code == 200:
        speakers = response.json()
        for speaker in speakers:
            name = speaker["name"]
            styles = [style["name"] for style in speaker["styles"]]
            ids = [style["id"] for style in speaker["styles"]]
            for id, style in zip(ids, styles):
                speakers_dict[id] = f"{name}:{style}"
                # print(f"Speaker: {name}, {style} id: {id}")
    else:
        print(f"Error: {response.status_code}")

    return speakers_dict


if __name__ == "__main__":
    speakers_dict = get_speakers()

In [187]:
id = random.randint(0, max(speakers_dict.keys()))
id = 5
name_style = speakers_dict[id]

In [188]:
def vvox_test(text, id):
    # エンジン起動時に表示されているIP、portを指定
    host = "127.0.0.1"
    port = 50021
    
    # 音声化する文言と話者を指定(3で標準ずんだもんになる)
    params = (
        ('text', text),
        ('speaker', id),
    )
    
    # 音声合成用のクエリ作成
    query = requests.post(
        f'http://{host}:{port}/audio_query',
        params=params
    )
    
    # 音声合成を実施
    synthesis = requests.post(
        f'http://{host}:{port}/synthesis',
        headers = {"Content-Type": "application/json"},
        params = params,
        data = json.dumps(query.json())
    )
    
    # 再生処理
    voice = synthesis.content
    pya = pyaudio.PyAudio()
    
    # サンプリングレートが24000以外だとずんだもんが高音になったり低音になったりする
    stream = pya.open(format=pyaudio.paInt16,
                      channels=1,
                      rate=24000,
                      output=True)
    
    stream.write(voice)
    stream.stop_stream()
    stream.close()
    pya.terminate()
    
if __name__ == "__main__":
    text = f"こんにちは。私の名前は{name_style}です。"
    vvox_test(text, id)